In [ ]:
# import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#초기에 넣어 줄 랜덤 넘버
batch_size = 1000
input_scale = 10
bins = 201
iteration = 100000
cost_graph=[]
iteration_graph=[]
input = tf.random_normal([batch_size,input_scale])

In [ ]:
#test용 input, potential
potential = tf.constant(0.0, shape = [batch_size,bins])

In [ ]:
#웨이트, 바이어스
'''간단하게 기존의 프로그램과 차이점은 기존의 프로그램에 두번째 스테이트를 쌓는 다는 점이다.'''
initializer = tf.contrib.layers.xavier_initializer()
# weight = tf.zeros([2, input_scale, bins], dtype = tf.float32)
# weight = tf.zeros([input_scale, bins], dtype = tf.float32)
weight1 = tf.Variable(initializer([input_scale, bins]), dtype = tf.float32)
bias1 = tf.Variable(initializer([bins]), dtype = tf.float32)
weight2 = tf.Variable(initializer([input_scale, bins]), dtype = tf.float32)
bias2 = tf.Variable(initializer([bins]), dtype = tf.float32)

In [ ]:
w1_loss, b1_loss, w2_loss, b2_loss = tf.nn.l2_loss(weight1), tf.nn.l2_loss(bias1), tf.nn.l2_loss(weight2), tf.nn.l2_loss(bias2)

In [ ]:
binsnorm = tf.sqrt(tf.cast(bins, tf.float32))
#프사이 1
psi_1 = tf.matmul(input, weight1) + bias1
psi_1 = tf.divide(psi_1,tf.sqrt(tf.reduce_mean(tf.square(psi_1), axis = 1, keepdims = True)))/binsnorm

## 에너지 1
zeroten = tf.zeros([batch_size, 1], tf.float32)
psil_1 = tf.concat([psi_1[:,1:],zeroten],1)
psir_1 = tf.concat([zeroten,psi_1[:,:-1]],1)
energy_1 = tf.reduce_mean(tf.subtract(tf.multiply(tf.square(psi_1*binsnorm),tf.add(potential,1.*bins*bins)),tf.multiply(tf.multiply(tf.add(psil_1*binsnorm,psir_1*binsnorm),psi_1*binsnorm),0.5*bins*bins)), axis=-1, keep_dims=True)

## 그람슈미츠 직교화
'''그람슈미츠 직교화를 통해 프사이1과 orthonormal한 프사이2를 만드는 과정이다.'''
psi_2 = tf.matmul(input, weight2) + bias2
dotprod_1_2 = tf.multiply(tf.reduce_sum(tf.multiply(psi_1, psi_2), axis = 1, keepdims = True), psi_1)
psi_2 = psi_2-dotprod_1_2
psi_2 = tf.divide(psi_2,tf.sqrt(tf.reduce_mean(tf.square(psi_2), axis = 1, keepdims = True)))/binsnorm


psil_2 = tf.concat([psi_2[:,1:],zeroten],1)
psir_2 = tf.concat([zeroten,psi_2[:,:-1]],1)
energy_2 = tf.reduce_mean(tf.subtract(tf.multiply(tf.square(psi_2*binsnorm),tf.add(potential,1.*bins*bins)),tf.multiply(tf.multiply(tf.add(psil_2*binsnorm,psir_2*binsnorm),psi_2*binsnorm),0.5*bins*bins)), axis=-1, keep_dims=True)

## Leanier combination
'''사실 이 계수는 별로 중요하지 않다. 왜냐하면 두 파동함수가 직교하기 때문에
첫번째 프사이가 그라운드 스테이트로 간 후,
두번째 프사이가 그라운드 스테이트로 가기 위해서 무조건 첫번째 들뜬 상태로 간다'''
C=1/tf.sqrt(2.)
C_1 = C*tf.ones(shape=[batch_size, 1])
C_2 = C*tf.ones(shape=[batch_size, 1])
psi = tf.add(tf.multiply(C_1, psi_1), tf.multiply(C_2, psi_2))

In [ ]:
total_energy = (C_1**2)*energy_1+(C_2**2)*energy_2
# total_energy = energy_1
# cost = tf.reduce_mean(total_energy)
cost = tf.reduce_mean(total_energy)+0.02*(w1_loss+w2_loss)

In [ ]:
'''여기선 varlist라는 개념을 사용했는데 이는 텐서플로가 변수로 인식하는 부분을 의미한다.
첫번째 학습에서는 웨이트1과 바이어스1만이 텐서플로의 변수이고
두번째 학습에서는 웨이트2와 바이어스2만이 변수로 작용한다.'''
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(energy_1, var_list=[weight1, bias1])
train2 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost, var_list=[weight2, bias2])

In [ ]:
'''생각보다 프로그램이 무거워져서 GPU를 사용해야 빠르게 돌아간다.'''
# If you want to use CPU
config = tf.ConfigProto(device_count = {'GPU': 0})

sess = tf.Session(config=config)

# If you want to use GPU
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# sess = tf.Session(config=config)

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
'''기존으 프로그램을 통해 이터레이션이 약 20000정도 진행되면 충분히 그라운드 스테이트에 도달한다는 점을
착안하여, 30000번 쯤을 기준으로 학습1에서 학습2로 넘어가게 설정했다.
파란색은 그라운드, 초록색은 퍼스트, 빨간색은 이 둘의 합을 나타내는데
퍼스트가 한쪽으로 치우친 두가지형태가 있기 때문에 둘의 합도 한쪽으로 치우치게 된다.
그림에서 퍼스트가 대칭인 이유는 제곱이기 때문.'''

k = 50

list_psi = np.zeros([int(iteration/k)+1, bins, 3])
list_energy = np.zeros([int(iteration/k)+1, 2])
list_cost = np.zeros([int(iteration/k)+1, 1])
n = 0
tic = time.time()

for i in range (iteration):  
    if i < 50000:
        sess.run(train)
    else :
        sess.run(train2)
    # save data each 50 times
    if (i > 0) & (i % k == 0):
        psi0_now, psi1_now, psi2_now, cost_now, energy1_now, energy2_now = sess.run([
            psi, psi_1, psi_2, cost, tf.reduce_mean(energy_1), tf.reduce_mean(energy_2)])
        list_psi[n, :, 0] = psi0_now[0]
        list_psi[n, :, 1] = psi1_now[0]
        list_psi[n, :, 2] = psi2_now[0]
        list_cost[n] = cost_now
        list_energy[n, 0] = energy1_now
        list_energy[n, 1] = energy2_now
        n = n + 1
        
    if (i > 0) & (i % 2000 == 0):
        print("iteration=%d, cost=%f, energy1=%f, energy2=%f" % (i, cost_now, energy1_now, energy2_now))
print('Works done, It take %.2f minutes'%((time.time() - tic)/60))
energy1_now, energy2_now = sess.run([tf.reduce_mean(energy_1), tf.reduce_mean(energy_2)])

In [ ]:
name = ['3_psi_total_exc.csv', '3_psi_1_exc.csv', '3_psi_2_exc.csv', '3_cost_exc.csv']

psi_total = list_psi[:, :, 0]
psi_1 = list_psi[:, :, 1]
psi_2 = list_psi[:, :, 2]
df = list()
df.append(pd.DataFrame(psi_total))
df.append(pd.DataFrame(psi_1))
df.append(pd.DataFrame(psi_2))
df.append(pd.DataFrame(list_cost))

for i in range(len(df)):
    df[i].to_csv(name[i])

In [ ]:
# Use Numerov method to solve schrodinger's equation
def numerov_schro(V, C=1, xr=[0, 1], n=4):
    bins = len(V)
    # C = h^2/(2m)
    x = np.linspace(xr[0], xr[1], bins)
    # dx = x[1] - x[0] -> (xr[1] - xr[0])/bins
    N = (xr[1] - xr[0]) * bins
    # make second derivative bins x bins matrix
    # make potential matrix
    pot = np.diag(V)
    # second derivative
    temp = np.ones([bins])
    temp = np.diag(-2 * temp)
    for i in range(bins-1):
        temp[i, i+1] = 1
        temp[i+1, i] = 1
    # sdmat = -C/dx**2 * temp
    sdmat = -1/2 * temp * N**2
    A = sdmat + pot
    # solve eigenvalue problem
    [ee, ev] = np.linalg.eig(A)
    [ee, ev] = arange_eig(ee, ev, n)
    return ee, ev

def arange_eig(ee, ev, n):
    # returns k in small order
    ee = list(ee); ev = list(ev.transpose())
    value = list(); vector = list()
    for sm in range(n):
        emin = 10e8 
        for i, val in enumerate(ee):
            if (emin > val):
                emin = val
                index = i
            elif val < 0:
                continue
        value.append(emin)
        vector.append(ev[index])
        ee.pop(index)
        ev.pop(index)
    return np.array(value), np.array(vector)

def compare_psi(psi, pot, state=1):
    # compare trained data with numerically calculated data
    # psi: trained data
    # pot: potential that you used to train data
    # state: the number of excited state
    [_, ee] = numerov_schro(pot)
    psi_num = ee[state - 1]
    # compare the data after taking the square
    # use chi square test, H0: trained data == expected data
    observed = psi**2/sum(psi**2); expected = psi_num**2/sum(psi_num**2)
    # plt.plot(observed); plt.plot(expected); plt.show()
    chival = np.sum((observed - expected)**2/expected)
    # pvalue = 1 - stats.chi2.cdf(chival, len(psi)-1)
    return chival

In [ ]:
pot = np.zeros([bins])
psi1 = psi1_now[0]
psi2 = psi2_now[0]
[ee, ev] = numerov_schro(pot)

In [ ]:
fig = plt.figure(figsize=(20, 30))

n = np.linspace(0, iteration, len(list_cost))

plt.subplot(3,1,1)
plt.title('cost', fontsize=18)
plt.plot(n[:-2], list_cost[:-2], 'k'); plt.grid(True)
plt.xlabel('iteration', fontsize=12)
plt.ylabel('cost', fontsize=12)
#plt.ylim([20000, 0000])

plt.subplot(3,1,2)
plt.title('cost', fontsize=18)
plt.plot(n[:-2], list_cost[:-2], 'k'); plt.grid(True)
plt.xlabel('iteration', fontsize=12)
plt.ylabel('cost', fontsize=12)
plt.ylim([-10, 100])

x = np.linspace(0, 1, bins)
pot = np.zeros(len(x))

plt.subplot(3,2,5)
plt.title('ground state', fontsize=18)
plt.plot(x, psi1**2, 'r', label='trained')
plt.plot(x, ev[0]**2, 'b', label='numerical')
plt.xlabel('x', fontsize=12)
plt.ylabel('probability', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)

[ee, ev] = numerov_schro(pot)
plt.subplot(3,2,6)
plt.title('1st excited state', fontsize=18)
plt.plot(x, psi2**2, 'r', label='trained')
plt.plot(x, ev[1]**2, 'b', label='numerical')
plt.xlabel('x', fontsize=12)
plt.ylabel('probability', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)

In [ ]:
error1 = np.sum((psi1**2 - ev[0]**2)**2*100)
error2 = np.sum((psi2**2 - ev[1]**2)**2*100)
print('ground state error = %.2f, 1st excited state error = %.2f'%(error1, error2))
print('ground state total energy = %.2f, 1st excited state total energy = %.2f'%(energy1_now, energy2_now))